# Demonstrator $M-\kappa$ analysis

First estimate of the flexural capacity of critical cross section OriSheG - used as a demonstrator object in TRR

 - For an estimated cross sectional height evaluate the flexural capacity

## Folded section #1

In [1]:
%matplotlib widget
import numpy as np
from bmcs_cross_section.api import ReinfLayer, BarLayer, MKappa, TShape
mc = MKappa(low_kappa=0, high_kappa=0.0001)
mc.cs_design.cross_section_shape_.trait_set(H=800, B=300)
mc.cs_design.concrete.compression = 'EC2 with plateau'

In [2]:
mc.cs_design.cross_section_shape = 'custom'
thickness = 15 # mm
H = 125 # mm
B_lower = 71.5 # mm
B_upper = 321.5 # 250 + B_lower # mm
#theta = np.pi / 2 * 1 / 2
theta = np.deg2rad(39.13)

In [3]:
gamma = (np.pi - theta) / 2
B_lower_top = B_lower - thickness / np.tan(gamma)

In [4]:
mc.cs_design.cross_section_shape_.cs_points = np.array([
    [B_lower_top, thickness],
    [0, thickness],
    [0, 0],
    [B_lower, 0],
    [B_lower + (H - thickness) / np.tan(theta), H - thickness ], 
    [B_lower + (H - thickness) / np.tan(theta) + B_upper, H - thickness ], 
    [B_lower + (H - thickness) / np.tan(theta) + B_upper, H ], 
    [B_lower_top + + (H - thickness) / np.tan(theta), H], 
])

In [5]:
mc.cs_design.cross_section_shape_.interact()

Output()

In [6]:
mc.cs_design.concrete.compression_.trait_set(
    f_c = 68.2, eps_cy=-0.0035, eps_cu=-0.0038, E_cc=24805
)

In [7]:
mc.cs_design.concrete.compression_.interact()

Output()

In [8]:
A_roving = (1.8+1.95)/2 # 3.62 # [mm**2] # Average of the range given on the catalogue
spacing = 12.7 # [mm] 
width = B_lower # [mm] 
E_carbon = 200000 # [MPa] 
n_rovings = width / spacing
A_layer = n_rovings * A_roving
n_rovings, A_layer

(5.62992125984252, 10.556102362204726)

In [9]:
_f_t = 1200
bl1 = ReinfLayer(name='carbon 1', matmod='carbon', z=5, A=A_layer)
bl1.matmod_.trait_set(f_t = _f_t, E=E_carbon)
bl2 = ReinfLayer(name='carbon 2', matmod='carbon', z=10, A=A_layer)
bl2.matmod_.trait_set(f_t = _f_t, E=E_carbon)

In [10]:
mc.cross_section_layout.add_layer(bl1)
mc.cross_section_layout.add_layer(bl2)

In [11]:
mc.high_kappa=0.0003
mc.interact()

Output()

in update plot 1e-07


### Moment-capacity of a single CRC module based on the iterative design calculations

In [12]:
# Multiply it by 2 to get the moment for the whole cross section, as only the half is considered in the analysis
M_R_kappa = abs(mc.M_t.max())*10**-6*2
M_R_kappa, 'kNm'

(5.688760919256367, 'kNm')

#### Moment-capacity of a single CRC waterbomb module from the experiments

In [13]:
# Max load from the tests
F_301 = 30.72 
F_303 = 26.18
F_304 = 29.92

# Distance between supports from the test (span)
L_support = 0.691

# Moment from the load test
M_301 = 1/4 * L_support * F_301 
M_303 = 1/4 * L_support * F_303 
M_304 =  1/4 * L_support* F_304 

# Moment from self weight one waterbomb is too small and can be neglected
M_301_weight = 1/8 * 0.691**2 * 0.2732 / 1.1085 # Area load to line load conversion

M_301, M_303, M_304, 'kNm'

(5.30688, 4.522595, 5.16868, 'kNm')

#### Load-capacity of an assembly of CRC waterbomb modules based usign the results from the iterative design calculations

![title](demonstrator-oben.png)
![title](demonstrator-beam.png)


In [14]:
# Without self weight

# Demonstrator: Assembly with 4 modules on the sides and 3 on the middle

L_assembly = 3.53 # m
B_assembly = 2.98 # m
F_1_assembly = 4 * M_301 / L_assembly * 2 / 10 # multiply by 2, there are two waterbomb elements on the cross-section of the assembly
F_3_assembly = 4 * M_303 / L_assembly * 2 / 10
F_4_assembly = 4 * M_304 / L_assembly * 2 / 10

F_1_assembly, F_3_assembly, F_4_assembly, 'ton'

(1.2026923512747874, 1.0249507082152975, 1.17137223796034, 'ton')

#### Self weight of the assembly

In [15]:
# Concrete density
rho_ton_m3 = 2.400 # t / m3  
rho_kN_m3 = rho_ton_m3 * 10
rho_kN_m3, 'kN/m3'

(24.0, 'kN/m3')

In [16]:
# Cross-section area
A_c_mm2 = mc.cs_design.cross_section_shape_.get_cs_area()
A_c_module_m2 = A_c_mm2 / (1000**2) * 2 # Multiply by 2 to consider the another half of the cross-section
A_c_assembly_m2 = A_c_module_m2 * 2 # Multiply by 2, the cross-section of the assembly consists of two modules
A_c_module_m2, A_c_assembly_m2, 'm2'

(0.01701911396171042, 0.03403822792342084, 'm2')

In [17]:
V_rhino_assembly = 184350540 * 10**-9 # m3 # Volume of assembly from the rhino model considering web, openings and half elements on both sides
V_rhino_element = 13228572.4 * 10**-9 # m3 # Volume of one element from the rhino model considering web, openings and half elements

w_assembly = V_rhino_assembly * rho_kN_m3
w_element = V_rhino_element * rho_kN_m3

g_assembly = rho_kN_m3 * A_c_assembly_m2 
# waterbomb elements on the cross-section of the assembly

w_assembly, w_element,'kN', g_assembly, 'kN/m' # It matches the experimental value, the mass of a single waterbomb module is 27~30 kg

(4.42441296, 0.31748573760000004, 'kN', 0.8169174701621, 'kN/m')

In [18]:
## It underestimates the weight
#V_m = A_c_m2 * L_assembly # considering ideal cross-section over the slab length
#m_kN = rho_kN_m3 * V_m * 2 # two interlocked layers
#m_kN, rho_kN_m3 * A_c_m2

In [19]:
# Moment from the self weight
M_self_weight = g_assembly * L_assembly**2 / 8
M_self_weight, 'kNm'

(1.2724408629928639, 'kNm')

In [20]:
# Load capacity considering the self-weight
w_assembly_ton = w_assembly / 10
F_1_total = F_1_assembly + w_assembly_ton
F_3_total = F_3_assembly + w_assembly_ton
F_4_total = F_4_assembly + w_assembly_ton

F_1_total, F_3_total, F_4_total, 'ton'

(1.6451336472747873, 1.4673920042152975, 1.6138135339603399, 'ton')

In [21]:
# Load-capacity/weight ratio of the assembly
F_1_total / w_assembly_ton, F_3_total / w_assembly_ton, F_4_total / w_assembly_ton

(3.718309439349412, 3.316580114653894, 3.647520131033022)

In [22]:
# Reinforcement ratio of the waterbomb assembly

reinf_ratio_assembly = A_layer*2*2*10**-4 / A_c_assembly_m2 # Multiply the reinforcement area by 2 to consider the two layers and by 2 to consider the two modules
reinf_ratio_assembly

0.12404996389299505

In [23]:
#M_mu_kNm = 4.28 # kNm

$$
 R = \frac{F}{2} \\
 M = R \frac{L}{2} \\
 M = \frac{1}{4} FL \\
 F = 4 \frac{M}{L}
$$

In [24]:
#F_mu_kN = 4 * M_mu_kNm / L_assembly
#F_mu_kN

In [25]:
#F_mu_kN / m_kN

# Rectangular cross section

In [26]:
%matplotlib widget
import numpy as np
from bmcs_cross_section.api import ReinfLayer, BarLayer, MKappa, TShape
mc_rec_carbon= MKappa(low_kappa=0, high_kappa=0.00002)
mc_rec_carbon.cs_design.cross_section_shape_.trait_set(H=135, B=B_assembly*1000) # Convert from m to mm
mc_rec_carbon.cs_design.concrete.compression = 'EC2 with plateau'

In [27]:
A_roving = (1.8+1.95)/2 # 3.62 # [mm**2] # Average of the range given on the catalogue
spacing = 12.7 # [mm] 
width = B_assembly*1000 # [mm] 
E_carbon = 200000 # [MPa] 
n_rovings = width / spacing
A_layer = n_rovings * A_roving
n_rovings, A_layer

(234.6456692913386, 439.9606299212599)

In [28]:
mc_rec_carbon.cs_design.concrete.compression_.trait_set(
    f_c = 68.2, eps_cy=-0.0035, eps_cu=-0.0038, E_cc=24805
)
_f_t = 1200
bl1 = ReinfLayer(name='carbon 1', matmod='carbon', z=5, A=A_layer)
bl1.matmod_.trait_set(f_t = _f_t, E=E_carbon)
bl2 = ReinfLayer(name='carbon 2', matmod='carbon', z=10, A=A_layer)
bl2.matmod_.trait_set(f_t = _f_t, E=E_carbon)
A_layer

439.9606299212599

In [29]:

mc_rec_carbon.cross_section_layout.add_layer(bl1)
mc_rec_carbon.interact()

Output()

in update plot 1e-07


In [30]:
# M_R from the carbon rectangular slab
M_R_carbon_rec = abs(mc_rec_carbon.M_t.max())*10**-6
M_R_carbon_rec, 'kNm'

(37.584624255033624, 'kNm')

In [31]:
# Self weight from the carbon rectangular slab
w_carbon_rec = rho_kN_m3 * H/1000 * B_assembly  * L_assembly/ 10
w_carbon_rec, 'ton', H/1000, B_assembly, L_assembly,  'm'

(3.1558199999999994, 'ton', 0.125, 2.98, 3.53, 'm')

In [32]:
F_carbon_rec = 4 * M_R_carbon_rec / L_assembly / 10
F_carbon_rec, 'ton'

(4.258880935414576, 'ton')

In [33]:
F_carbon_rec / w_carbon_rec

1.349532272250818